In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_two_year'].values

In [3]:
#### CART
depth = [8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                          learning_rate=learning_rate, 
                          depth=depth, 
                          estimators=estimators, 
                          seed=816)

In [4]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.5948129723221217, 0.5891675512391937)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_two_year'].values

## columns 
cols = KY_X.columns

In [11]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.005, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge19.0',
 'age_at_current_charge20.0',
 'age_at_current_charge21.0',
 'age_at_current_charge24.0',
 'age_at_current_charge26.0',
 'age_at_current_charge29.0',
 'age_at_current_charge30.0',
 'age_at_current_charge35.0',
 'age_at_current_charge38.0',
 'age_at_current_charge39.0',
 'age_at_current_charge44.0',
 'age_at_current_charge45.0',
 'age_at_current_charge46.0',
 'age_at_current_charge48.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_arrest7',
 'p_arrest8',
 'p_charges1',
 'p_violence1',
 'p_felony1',
 'p_felony3',
 'p_misdemeanor1',
 'p_property3',
 'p_drug2',
 'p_stealing1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'p_pending_charge2',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'five_year1']

In [12]:
c_grid={'C': [0.001, 0.002, 0.003, 0.004, 0.005]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [14]:
stumps_summary['best_params'], stumps_summary['FL_score'], stumps_summary['auc_diffs']

({'C': 0.005}, 0.598181213263835, 0.0003255805107428289)

### RiskSLIM

In [15]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_two_year'].values

## columns 
cols = KY_X.columns

In [23]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 
                                        0.0008, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

15

In [24]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [25]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 02:51 PM | 203 rows in lookup table
11/12/19 @ 02:51 PM | ------------------------------------------------------------
11/12/19 @ 02:51 PM | runnning initialization procedure
11/12/19 @ 02:51 PM | ------------------------------------------------------------
11/12/19 @ 02:51 PM | CPA produced 2 cuts
11/12/19 @ 02:51 PM | running naive rounding on 4 solutions
11/12/19 @ 02:51 PM | best objective value: 0.6931
11/12/19 @ 02:51 PM | rounding produced 4 integer solutions
11/12/19 @ 02:51 PM | best objective value is 0.6931
11/12/19 @ 02:51 PM | running sequential rounding on 4 solutions
11/12/19 @ 02:51 PM | best objective value: 0.6931
11/12/19 @ 02:51 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 02:51 PM | polishing 4 solutions
11/12/19 @ 02:51 PM | best objective value: 0.6931
11/12/19 @ 02:51 PM | polishing produced 4 integer solutions
11/12/19 @ 02:51 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:51 PM | adding 210 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5481.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 363911 77463        0.5428    11        0.5439        0.5303  2005327    2.50%           rho_6 D 363911 363910     40
 372331 77658        0.5335     2        0.5439        0.5306  2036155    2.45%           rho_9 D 372331 372329     25
 380570 78380        0.5435    15        0.5439        0.5309  2065034    2.39%           rho_4 D 380570 380569     29
 388600 77311        0.5436     8        0.5439        0.5312  2094064    2.33%        alpha_14 D 388600 388599     23
 396845 75942        0.5316    12        0.5439        0.5316  2124184    2.26%           rho_3 U 396845 396840     26
 405522 75635        cutoff              0.5439        0.5320  2152605    2.20%          rho_12 U 405522 405520     28
 412930 75105        cutoff              0.5439        0.5322  2181414    2.16%          rho_12 U 412930 125292     50
 419967 73171        cutoff              0.5439        0.5324  2210304    2.11%        alpha_14 U 419967 419965     36
Elapsed time = 67.81 sec. (50790.39 ticks, tree 

11/12/19 @ 02:53 PM | 203 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.52909697603445749
CPXPARAM_MIP_Tolerances_UpperCutoff              0.55775000546001796


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:53 PM | adding 231 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5578.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:57 PM | adding 216 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5473.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 286924 69399        cutoff              0.5473        0.5319  1326045    2.81%           rho_9 U 286924 286923     33
 293660 69628        0.5431    10        0.5473        0.5321  1348566    2.78%          rho_12 U 293660 293659     24
 300365 70485        cutoff              0.5473        0.5321  1368668    2.77%         alpha_5 U 300365 229769     33
 306529 69948        0.5394    11        0.5473        0.5323  1389742    2.73%          rho_12 D 306529 306528     21
 312415 70381        0.5406     6        0.5473        0.5326  1411732    2.69%          rho_10 D 312415 312413     34
 318364 70467        0.5400     8        0.5473        0.5328  1433120    2.65%           rho_3 U 318364  61820     27
 325106 71352        0.5425    11        0.5473        0.5330  1454209    2.62%           rho_3 D 325106 325104     26
 331350 71020        0.5352     2        0.5473        0.5331  1474291    2.59%           rho_9 D 331350 331348     36
Elapsed time = 34.38 sec. (50794.86 ticks, tree 

11/12/19 @ 02:59 PM | ------------------------------------------------------------
11/12/19 @ 02:59 PM | CPA produced 2 cuts
11/12/19 @ 02:59 PM | running naive rounding on 4 solutions
11/12/19 @ 02:59 PM | best objective value: 0.6931
11/12/19 @ 02:59 PM | rounding produced 4 integer solutions
11/12/19 @ 02:59 PM | best objective value is 0.6931
11/12/19 @ 02:59 PM | running sequential rounding on 4 solutions
11/12/19 @ 02:59 PM | best objective value: 0.6931
11/12/19 @ 02:59 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 02:59 PM | polishing 4 solutions
11/12/19 @ 02:59 PM | best objective value: 0.6931
11/12/19 @ 02:59 PM | polishing produced 4 integer solutions
11/12/19 @ 02:59 PM | initialization produced 7 feasible solutions
11/12/19 @ 02:59 PM | best objective value: 0.5481
11/12/19 @ 02:59 PM | ------------------------------------------------------------
11/12/19 @ 02:59 PM | completed initialization procedure
11/12/19 @ 02:59 PM | ----------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:59 PM | adding 211 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5481.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 281343 70288        cutoff              0.5481        0.5294  1888839    3.41%          rho_15 D 281343 281342     25
 286741 71484        0.5377    11        0.5481        0.5294  1929378    3.41%           rho_6 D 286741 286739     19
 292311 71893        0.5456    12        0.5481        0.5294  1974299    3.41%           rho_9 D 292311 292281     33
 297797 72243        cutoff              0.5481        0.5294  2015886    3.41%           rho_9 U 297797 297796     33
 303071 73509        0.5303    13        0.5481        0.5294  2055759    3.41%           rho_4 N 303071 303070     17
 308808 75305        0.5444     9        0.5481        0.5295  2094636    3.39%           rho_4 D 308808 308806     28
 316282 76579        0.5320    17        0.5481        0.5301  2123223    3.28%           rho_4 D 316282 316281     17
 323557 78073        0.5323    13        0.5481        0.5304  2150336    3.22%           rho_9 D 323557 323556     17
Elapsed time = 52.19 sec. (50791.32 ticks, tree 

In [26]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_validation_score'])

(0.579126200774199, 0.7069996106482022)

In [27]:
#### save results
summary_general2_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}

In [28]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["", "CART", "EBM", "Lasso Stumps", "RiskSLIM"],
           ["General",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)